## Home work 6

In [12]:
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from typing import Tuple
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from mlflow.models import infer_signature
import mlflow

In [13]:
import pickle

In [14]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


In [4]:
catalog


{'dataset_id_214': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/01_raw/dataset_id_214.csv'), "
                   "protocol='file', load_args={}, save_args={'index': False})",
 'inference_data': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/01_raw/sampled_file.csv'), "
                   "protocol='file', load_args={}, save_args={'index': False})",
 'encoder': "kedro_datasets.pickle.pickle_dataset.PickleDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/03_primary/encoder.pkl'), "
            "backend='pickle', protocol='file', load_args={}, save_args={})",
 'preprocessed_data': "kedro_datasets.pandas.csv_dataset.CSVDataset(filepath=PurePosixPath('C:/Users/Admin/Desktop/AI-839/srinivasan-ai-839/data/02_modelinput/preprocessed_data.csv'), "
                      "protocol='file', load_args={}, save_a

In [15]:
def preprocess_data(df: pd.DataFrame,encoder:OneHotEncoder) -> Tuple[pd.DataFrame]:
    """
    Preprocesses the input DataFrame by converting categorical columns into one-hot encoded columns
    using sklearn's OneHotEncoder. The numerical columns are retained and concatenated with the
    encoded categorical columns.

    Parameters:
    -----------
    df : pd.DataFrame
        The input DataFrame containing both categorical and numerical columns.
    encoder: OneHotEncoder
        The one hot encoder that has been stored in the configuration
    Returns:
    --------
    Tuple[pd.DataFrame, OneHotEncoder]
        - A DataFrame with categorical columns one-hot encoded and numerical columns retained.
        # - The fitted OneHotEncoder instance.
    """
    # Identify numeric and categorical columns
    num_cols = df._get_numeric_data()
    cat_cols = df.select_dtypes(include=['object'])
    
    if not cat_cols.empty:
        # Initialize and fit the OneHotEncoder on categorical columns
        # encoder = OneHotEncoder(sparse_output=False, dtype='float', drop=None)
        cat_cols_ohe = encoder.fit_transform(cat_cols)
        
        # Convert the encoded columns into a DataFrame
        cat_cols_ohe_df = pd.DataFrame(
            cat_cols_ohe, columns=encoder.get_feature_names_out(cat_cols.columns)
        )
        
        # Concatenate the one-hot encoded categorical columns with the numeric columns
        df_processed = pd.concat([cat_cols_ohe_df, num_cols.reset_index(drop=True)], axis=1)
    else:
        df_processed = df
        encoder = None

    # Save the processed DataFrame to a CSV file
    # df_processed.to_csv("C:\\Users\\Admin\\Desktop\\AI-839\\srinivasan-ai-839\\data\\01_raw\\pre_processed.csv", index=False)
    
    return df_processed

In [16]:
encoder = catalog.load("encoder")

[10/16/24 22:02:36] INFO     Loading data from encoder (PickleDataset)...                       ]8;id=820744;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=926790;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [17]:
dataset = catalog.load("preprocessed_data")

[10/16/24 22:02:39] INFO     Loading data from preprocessed_data (CSVDataset)...                ]8;id=315762;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=383097;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [18]:
model = catalog.load("model2")

[10/16/24 22:02:41] INFO     Loading data from model2 (PickleDataset)...                        ]8;id=234422;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=837279;file://c:\Users\Admin\miniconda3\envs\MlOps\lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [19]:
print(type(dataset))

<class 'pandas.core.frame.DataFrame'>


In [27]:
dataset_new = preprocess_data(dataset,encoder)

In [28]:
dataset_truncate = dataset_new[10:]

In [29]:
dataset_truncate.drop(columns=["Unnamed: 0"],inplace=True)
dataset_truncate


,checking_status_0<=X<200,checking_status_<0,checking_status_>=200,checking_status_no checking,credit_history_all paid,credit_history_critical/other existing credit,credit_history_delayed previously,credit_history_existing paid,credit_history_no credits/all paid,purpose_business,...,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,y
10,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.472068,0.419630,0.652240,0.945691,0.115385,0.871547,0.927647,0.664855,0.014704,True
11,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.424463,0.240885,0.958707,0.691337,0.691825,0.732606,0.748178,0.923704,0.786498,True
12,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.317776,0.400655,0.610133,0.528451,0.184162,0.786356,0.064903,0.429595,0.005475,False
13,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.552385,0.799394,0.859617,0.199796,0.091506,0.215622,0.830488,0.448498,0.492287,True
14,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.917147,0.128064,0.727007,0.199042,0.611148,0.545832,0.234905,0.553738,0.847184,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.882218,0.883794,0.233441,0.150932,0.499922,0.434244,0.140232,0.187078,0.141813,False
96,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.149197,0.043498,0.134882,0.350446,0.889813,0.760545,0.261638,0.823732,0.053394,True
97,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.424718,0.253165,0.242619,0.903971,0.694778,0.517654,0.707042,0.575040,0.073795,False
98,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.469677,0.654652,0.316479,0.831883,0.405854,0.259628,0.350143,0.868693,0.065413,True


In [30]:
X = dataset_truncate.drop(columns=["y"])

In [32]:
Y = dataset_truncate['y']

In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.2, random_state=42
    )

In [34]:
# model = RandomForestClassifier(random_state=42)
model.fit(X_train,Y_train)

RandomForestClassifier(random_state=42)

In [35]:
Y_pred = model.predict(X_test)

In [36]:
accuracy_score(Y_test,Y_pred)

0.9444444444444444

In [39]:
import pickle
with open("model_new.pkl", "wb") as f: 
    pickle.dump(model, f)

In [4]:
!cd .. && mlflow_automate.bat loan_pred_hw06


^C
Press any key to continue . . . 


2024/10/17 10:48:59 INFO mlflow.models.cli: Generating Dockerfile for model models:/loan_pred_hw06/latest
C:\Users\Admin\miniconda3\envs\MLOps\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])

2024/10/17 10:49:00 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
C:\Users\Admin\miniconda3\envs\MLOps\lib\site-packages\mlflow\store\artifact\utils\models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn m